In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [7]:
import requests

### Prototyping Scraping on Nicola Mirotic

In [4]:
url = "https://www.euroleague.net/competition/players/showplayer?pcode=000796&seasoncode=E2021#!currentstats"

In [8]:
html = requests.get(url)

In [14]:
html.text;

In [17]:
soup = BeautifulSoup(html.text, "html5lib")

In [51]:
soup

<!DOCTYPE html>
<html lang="en"><head>

    <script src="//ajax.googleapis.com/ajax/libs/jquery/1.11.1/jquery.min.js" type="text/javascript"></script>
    <script src="/aspnet_client/JavaScript/iepp.1-5-1.min.js" type="text/javascript"></script>
    <script src="//maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js" type="text/javascript"></script>
    <!-- DFP -->
    <script type="text/javascript">
        var googletag = googletag || {};
        googletag.cmd = googletag.cmd || [];
        (function () {
            var gads = document.createElement('script');
            gads.async = true;
            gads.type = 'text/javascript';
            var useSSL = 'https:' == document.location.protocol;
            gads.src = (useSSL ? 'https:' : 'http:') +
                '//www.googletagservices.com/tag/js/gpt.js';
            var node = document.getElementsByTagName('script')[0];
            node.parentNode.insertBefore(gads, node);
        })();
    </script>

    <link href="/

#### Grabbing player's name

In [90]:
player_data = soup.find_all(class_="player-data")

In [91]:
player_data[0].find_all(class_="name")[0].get_text()

'MIROTIC, NIKOLA'

#### Grabbing player's height and DOB

In [108]:
second_summary_spans = player_data[0].find_all(class_='summary-second')[0].find_all("span")

In [109]:
second_summary_spans[0].get_text().split(":")[1]

' 2.08'

In [110]:
second_summary_spans[1].get_text().split(":")[1]

' 11 February, 1991'

#### Grabbing player's position

In [96]:
first_summary_spans = player_data[0].find_all(class_="summary-first")[0].find_all("span")

In [106]:
first_summary_spans[1].find_all("span")[1].get_text()

'Forward'

#### Grabbing player's stats

In [73]:
stats_table = soup.find('table', id = "tblPlayerPhaseStatistics" )

In [85]:
stats_table.find_all(class_="PlayerGridRow")[0].find_all("td") # Stats

[<td class="PlayerTitleColumn">Totals</td>,
 <td>9</td>,
 <td>9</td>,
 <td>225:39</td>,
 <td>149</td>,
 <td>33/51</td>,
 <td>18/35</td>,
 <td>29/34</td>,
 <td>12</td>,
 <td>39</td>,
 <td>51</td>,
 <td>12</td>,
 <td>12</td>,
 <td>9</td>,
 <td>3</td>,
 <td>1</td>,
 <td>12</td>,
 <td>29</td>,
 <td>194</td>]

In [135]:
stats = stats_table.find_all(class_="PlayerGridRow")[0].find_all("td")

In [136]:
for stat in stats:
    print(stat.get_text())

Totals
9
9
225:39
149
33/51
18/35
29/34
12
39
51
12
12
9
3
1
12
29
194


In [137]:
len(stats)

19

In [131]:
headers = stats_table.find_all("tr")[1].find_all("th") # headers

In [132]:
for header in headers[1:]:
    print(header.get_text())

G
GS
Min
Pts
2FG
3FG
FT
O
D
T
As
St
To
Fv
Ag
Cm
Rv
PIR


In [138]:
len(headers)

19

#### Create dictionary with player's stats

In [190]:
def player_dictionary_creation(soup):
    player_data_dict = {}
    player_data = soup.find_all(class_="player-data")
    name_of_player = player_data[0].find_all(class_="name")[0].get_text().split(",")
    player_data_dict['name'] = name_of_player[1] + " " + name_of_player[0]
    first_summary_spans = player_data[0].find_all(class_="summary-first")[0].find_all("span")
    player_data_dict['position'] = first_summary_spans[1].find_all("span")[1].get_text()
    second_summary_spans = player_data[0].find_all(class_='summary-second')[0].find_all("span")
    player_data_dict['height'] = second_summary_spans[0].get_text().split(":")[1]
    player_data_dict['dob'] = second_summary_spans[1].get_text().split(":")[1]
    stats_table = soup.find('table', id = "tblPlayerPhaseStatistics" )
    stats = stats_table.find_all(class_="PlayerGridRow")[0].find_all("td")
    headers = stats_table.find_all("tr")[1].find_all("th") 
    for i, (stat, header) in enumerate(zip(stats,headers)):
        if i!=0:
            player_data_dict[header.get_text()] = stat.get_text()
    return player_data_dict

In [191]:
Mirotic_dict = player_dictionary_creation(soup)

In [192]:
Mirotic_dict

{'name': ' NIKOLA MIROTIC',
 'position': 'Forward',
 'height': ' 2.08',
 'dob': ' 11 February, 1991',
 'G': '9',
 'GS': '9',
 'Min': '225:39',
 'Pts': '149',
 '2FG': '33/51',
 '3FG': '18/35',
 'FT': '29/34',
 'O': '12',
 'D': '39',
 'T': '51',
 'As': '12',
 'St': '12',
 'To': '9',
 'Fv': '3',
 'Ag': '1',
 'Cm': '12',
 'Rv': '29',
 'PIR': '194'}